In [1]:
import hail as hl

In [2]:
hl.init()

Running on Apache Spark version 2.2.0
SparkUI available at http://10.1.7.107:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version devel-f2b0dca9f506
NOTE: This is a beta version. Interfaces may change
  during the beta period. We recommend pulling
  the latest changes weekly.


In [3]:
schema = {'variant': hl.tstr, 'rsid': hl.tstr, 'nCompleteSamples': hl.tint32,
         'AC': hl.tfloat64, 'ytx': hl.tfloat64, 'beta': hl.tfloat64,
         'se': hl.tfloat64, 'tstat': hl.tfloat64, 'pval': hl.tfloat64}

ht= hl.import_table('/Users/maccum/manhattan/50.assoc.tsv', key='variant', types=schema)

2018-08-09 11:53:15 Hail: INFO: Reading table with no type imputation
  Loading column 'variant' as type 'str' (user-specified)
  Loading column 'rsid' as type 'str' (user-specified)
  Loading column 'nCompleteSamples' as type 'int32' (user-specified)
  Loading column 'AC' as type 'float64' (user-specified)
  Loading column 'ytx' as type 'float64' (user-specified)
  Loading column 'beta' as type 'float64' (user-specified)
  Loading column 'se' as type 'float64' (user-specified)
  Loading column 'tstat' as type 'float64' (user-specified)
  Loading column 'pval' as type 'float64' (user-specified)



In [4]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'variant': str 
    'rsid': str 
    'nCompleteSamples': int32 
    'AC': float64 
    'ytx': float64 
    'beta': float64 
    'se': float64 
    'tstat': float64 
    'pval': float64 
----------------------------------------
Key: ['variant']
----------------------------------------


In [5]:
ht = ht.annotate(phenotype='standing_height')
ht = ht.transmute(**hl.parse_variant(ht.variant)).key_by('locus','alleles').drop('variant')

In [6]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'rsid': str 
    'nCompleteSamples': int32 
    'AC': float64 
    'ytx': float64 
    'beta': float64 
    'se': float64 
    'tstat': float64 
    'pval': float64 
    'phenotype': str 
    'locus': locus<GRCh37> 
    'alleles': array<str> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [7]:
mt = ht.distinct().to_matrix_table(row_key=['locus','alleles'],
                  col_key=['phenotype'],row_fields=['rsid'])

In [8]:
mt.write('/Users/maccum/manhattan_data/raw/standing_height_mt', overwrite=True)

2018-08-09 11:55:54 Hail: INFO: Ordering unsorted dataset with network shuffle
2018-08-09 12:00:38 Hail: INFO: Coerced sorted dataset
2018-08-09 12:03:13 Hail: INFO: wrote 10894596 items in 35 partitions to /Users/maccum/manhattan_data/raw/standing_height_mt


In [9]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'phenotype': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'rsid': str 
----------------------------------------
Entry fields:
    'nCompleteSamples': int32 
    'AC': float64 
    'ytx': float64 
    'beta': float64 
    'se': float64 
    'tstat': float64 
    'pval': float64 
----------------------------------------
Column key: ['phenotype']
Row key: ['locus', 'alleles']
Partition key: ['locus', 'alleles']
----------------------------------------
